## Ref.
- <https://stackoverflow.com/questions/32125281/removing-watermark-out-of-an-image-using-opencv>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

plt.style.use("dark_background")

In [ ]:
image_path = Path.cwd()/"figs/watermark.jpg"
#image_path = Path.cwd()/"figs/見本.png"
image_path.exists()

In [ ]:
bgr = cv2.imread(str(image_path))
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(rgb)
ax[0].set_title("rgb");
ax[1].imshow(gray, cmap="gray", vmin=0, vmax=255)
ax[1].set_title("gray");

In [ ]:
bg = gray.copy()

for r in range(1, 7):
    l = 2*r + 1
    kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE,
        (l, l),
    )
    bg = cv2.morphologyEx(bg, cv2.MORPH_CLOSE, kernel)
    bg = cv2.morphologyEx(bg, cv2.MORPH_OPEN, kernel)

plt.figure(figsize=(10,10))
plt.imshow(bg, cmap="gray", vmin=0, vmax=255)
plt.title("bg");

**(?)** Why `CLOSE` then `OPEN`? Any theory on this? What about `OPEN` then `CLOSE`?
Why we need to gradually increase the kernel size and run in a for loop?  
**(R)**   
- To help understand why the author used closing followed by opening, you could try
  to **comment the opening part** in the for loop above.  
    
  I have tried and found that keeping it as **closing followed opening is better**, because, although
  using only closing seems to be able to get rid of black-stroke texts in `bg`,
  these texts appear even **whiter than than even the white part** of `bg`.

Note that
- The circular shape thing is not as sharp as in the original image;
  besides, the black thin circumference line has been gotten rid of.
- The black texts inside the circular disappeared.

In [ ]:
print(f"{bg.shape, gray.shape, rgb.shape = }")
print(f"{bg.dtype, gray.dtype, rgb.dtype = }")

In [ ]:
diff = cv2.subtract(bg, gray)
plt.figure(figsize=(10,10))
plt.imshow(diff, vmin=0, vmax=255, cmap="gray")
plt.title("diff");

Let's briefly recall what `cv2.subtract` is capable of in the next cell:

In [ ]:
cv2.subtract(
    np.array([[200]], dtype=np.uint8),
    np.array([[255]], dtype=np.uint8)
)

In [ ]:
cv2.threshold?

In [ ]:
# bw for "Black and White"
bw = cv2.threshold(
    diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
dark = cv2.threshold(
    bg, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(bw, cmap="gray")
ax[0].set_title("bw");
ax[1].imshow(dark, cmap="gray")
ax[1].set_title("dark");

In [ ]:
np.unique(bw), np.unique(dark)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(bw, cmap="gray")
plt.title("bw");

In [ ]:
darkpix = gray[np.where(dark > 0)]
print(f"{darkpix.shape = }")
print(f"{np.product(gray.shape)}")
# plt.figure(figsize=(10,10))
# plt.imshow(darkpix, cmap="gray")
# plt.title("darkpix");

In [ ]:
np.where(dark > 0)

In [ ]:
darkpix = cv2.threshold(
    darkpix, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
print(f"{darkpix.shape = }")
plt.figure(figsize=(10,10))
plt.imshow(darkpix, cmap="gray")
plt.title("darkpix")

In [ ]:
bw[np.where(dark > 0)] = darkpix.T
plt.figure(figsize=(10,10))
plt.imshow(bw, cmap="gray")
plt.title("bw")

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(rgb)
plt.title("rgb");

In [ ]:
bw[np.where(dark > 0)].shape, darkpix.shape

**(?)** Could we get RGB with watermark removed instead of grayscale?  

## How Good Is The Final Result `bw`?

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cv2.subtract(gray, bw), vmin=0, vmax=255, cmap="gray")
plt.title("cv2.subtract(gray, bw)");

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cv2.subtract(bw, gray), vmin=0, vmax=255, cmap="gray")
plt.title("cv2.subtract(bw, gray)");

**(?)** Is `diff` binary like `bw` and `dark`?  
**(R)** No.

In [ ]:
np.unique(diff)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(diff, vmin=0, vmax=255, cmap="gray")
plt.title("diff");

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(bw, vmin=0, vmax=255, cmap="gray")
plt.title("bw");

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,10))
ax[0].imshow(rgb)
ax[0].set_title("rgb");
ax[1].imshow(bw, cmap="gray", vmin=0, vmax=255)
ax[1].set_title("bw");